In [ ]:
import numpy as np
from functions import *

# Preparation
init_nuc_num = 10000 # Initial number of each nucleobase
cleav_prop = 0.2 # Probability of breaking bonds in unstructured regions
cleav_prop_struct = 0.02 # Probability of breaking bonds in structured regions
length_threshold = 10 # Strands longer than this we will checked for structure
n_iterations = 100 # Number of iterations
progress_report_freq  = 10 # Number of iterations

# A general note: Here we use numpy array of dtype=object because Python
# can support arbitrarily large integers. In contrast, NumPy int64 type
# will overflow for long sequences
nucleotide_list = np.array([1, 2, 3, 4] * init_nuc_num, dtype=object)
# Mapping from integer numbers to nucleobase names
mapping = {"1": "A", "2": "G", "3": "C", "4": "U"}

# Initial Phase 1
# Shuffle the nucleobases, then pair them
# We represent a nucleic acid strand by a sequence of integers (e.g. 11 is AA)
# So multiply a number by 10 and to another number to create a dimer
np.random.shuffle(nucleotide_list)
size = nucleotide_list.size
nucleotide_list = nucleotide_list[:size//2] + 10 * nucleotide_list[size//2:]

# Initial Phase 3
# Randomly break dimer bonds to create a set of monomers and dimers
cleave = np.random.random(nucleotide_list.size) > cleav_prop
cleaved = nucleotide_list[cleave]
uncleaved = nucleotide_list[~cleave]

cleaved_1 = cleaved//10
cleaved_2 = cleaved%10

nucleotide_list = np.concatenate((uncleaved, cleaved_1, cleaved_2))

# Loop over the number of iterations
for it in range(1, n_iterations + 1):
    print(it)

    # Phase 1: Pair nucleotide strands
    # Shuffle list of nucleotide strands
    np.random.shuffle(nucleotide_list)

    # Take the first half of the strands and calculate their lengths
    size = nucleotide_list.size
    # Use dtype=object to keep arbitrary integer prevision
    order = order_of_mag(nucleotide_list[:size//2]).astype(object)

    # If we have an even number, pair all of them
    if size%2 == 0:
        nucleotide_list = nucleotide_list[:size//2] + 10**order * nucleotide_list[size//2:]
    # If we have an odd number, do not pair last strand
    else:
        nucleotide_list_temp = nucleotide_list[:size//2] + 10**order * nucleotide_list[size//2:-1]
        nucleotide_list = np.hstack((nucleotide_list_temp, nucleotide_list[-1]))

    # Phase 2: Determine folded structures and randomly break bonds in long strands
    order = order_of_mag(nucleotide_list).astype(object)
    mono = nucleotide_list[order == 1]
    short = nucleotide_list[np.logical_and(order > 1, order < length_threshold)]
    long = nucleotide_list[order >= length_threshold]

    long = break_long(long, cleav_prop, cleav_prop_struct, mapping)

    # Phase 3: Randomly break bonds in short strands
    short = break_short(short, cleav_prop)

    nucleotide_list = np.concatenate((mono, short, long))


    # Print progress report: show sequences of 10 longest strands
    if it%progress_report_freq==0:
        print("\nRESULTS\n")
        summary = np.sort(nucleotide_list)[::-1][:10]
        summary_s = convert_int_to_str_seq(summary, mapping)

        for s in summary_s:
            print(s)

1
2
3
4
5
6
7
8
9
10

RESULTS

GCGUCGAAAUGCCAACUGACGGGAGUUA
GGAAUUCGGGCAUCAACCUGCGAAAU
UCGCGCAGCAGCGCGACAUUCAUAG
CCCGCUUCCGGUCAGACACUCGCAC
UUCACGGCUACUACAUGAACGUCU
CGGGGCGUAUGUGCGACGGAUCGG
CAUUACUUGAUCGUAUAGUGUAA
GCUCAGCGCCGCUCACUGACUGC
AUUAGGGUCGCAUGAAGUCUGUC
UUCGUGAGCCCACUGCUUUGAA
11
12
13
14
15
16
17
18
19
20

RESULTS

CGUUUAUAUGCACGUCUGCGCGAGCCAGCCGUAC
GGUACGGCUAAUACUUAGGGCGGUGUCCCCCUG
UUUCUUUCACAGAACUUGUUGUGCAAAAGAA
CCUACGCUGGACAACGGAUGGGUCCGGCGG
GGCUCGGUGACAUAGACGCCCUCCCAGCUU
GCCGUUAGACUAGCCUAAUGUCUGAACUU
UCACACGCGCGCAGCGCUCAGCGUAGA
CGAAUUGAUCCUAUUUGCACCAGUUG
CCUAACAAGGUACCCCUAGUUCGCA
CGGAUCUCCGGUCUCUGACCGCCCA
21
22
23
